In [19]:
import boto3 

my_session = boto3.session.Session()
my_region = my_session.region_name
client = boto3.client("sts")
account_id = client.get_caller_identity()["Account"]
algorithm_name = "informer"

In [40]:
!./build_and_push.sh

230755935769
us-west-2
WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
base_img:763104351884.dkr.ecr.us-west-2.amazonaws.com/pytorch-training:1.7.1-gpu-py36-cu110-ubuntu18.04
Sending build context to Docker daemon  388.3MB
Step 1/7 : ARG BASE_IMG=${BASE_IMG}
Step 2/7 : FROM ${BASE_IMG}
 ---> 910c070760f2
Step 3/7 : ENV PATH="/opt/ml/code:${PATH}"
 ---> Running in c4426cab78ce
Removing intermediate container c4426ca

In [41]:
image_uri=f"{account_id}.dkr.ecr.{my_region}.amazonaws.com/{algorithm_name}"
image_uri

'230755935769.dkr.ecr.us-west-2.amazonaws.com/informer'

In [3]:
!git clone https://github.com/zhouhaoyi/ETDataset.git 

Cloning into 'ETDataset'...
remote: Enumerating objects: 187, done.
remote: Counting objects: 100% (187/187), done.
remote: Compressing objects: 100% (184/184), done.
remote: Total 187 (delta 66), reused 13 (delta 2), pack-reused 0
Receiving objects: 100% (187/187), 3.85 MiB | 10.36 MiB/s, done.
Resolving deltas: 100% (66/66), done.


In [4]:
!mkdir -p input
!mkdir -p input/data
!mkdir -p input/data/training
!mv ETDataset/ETT-small/*.csv input/data/training/


In [42]:
import sagemaker 
from sagemaker import get_execution_role
session = sagemaker.session.Session()
bucket = session.default_bucket()
role = get_execution_role()

In [43]:
s3_path = f"s3://{bucket}/data/ETH"
s3_path

's3://sagemaker-us-west-2-230755935769/data/ETH'

In [44]:
!aws s3 cp --recursive ./input/data/training/ $s3_path

upload: input/data/training/ETTh2.csv to s3://sagemaker-us-west-2-230755935769/data/ETH/ETTh2.csv
upload: input/data/training/ETTh1.csv to s3://sagemaker-us-west-2-230755935769/data/ETH/ETTh1.csv
upload: input/data/training/ETTm2.csv to s3://sagemaker-us-west-2-230755935769/data/ETH/ETTm2.csv
upload: input/data/training/ETTm1.csv to s3://sagemaker-us-west-2-230755935769/data/ETH/ETTm1.csv


In [45]:
from datetime import datetime
now = datetime.now()
timestamp = datetime.timestamp(now)
job_name = "informer-{}".format(str(int(timestamp))) 
job_name

'informer-1623914874'

In [46]:
mode = 'remote'
if mode == 'local':
    csess = sagemaker.local.LocalSession()
else:    
    csess = session

print(csess)
estimator = sagemaker.estimator.Estimator( 
                        role=role,
                        image_uri=image_uri,
                        instance_count=1,
#                         instance_type='local_gpu',
                        instance_type='ml.p2.xlarge',
                        sagemaker_session=csess,
                        volume_size=100, 
                        debugger_hook_config=False
                   )

In [47]:
estimator.fit(inputs={"training":s3_path}, job_name=job_name)

2021-06-17 07:27:58 Starting - Starting the training job...
2021-06-17 07:28:27 Starting - Launching requested ML instancesProfilerReport-1623914878: InProgress
......
2021-06-17 07:29:27 Starting - Preparing the instances for training.........
2021-06-17 07:30:54 Downloading - Downloading input data
2021-06-17 07:30:54 Training - Downloading the training image...................................{'model': 'informer', 'data': 'ETTh1', 'root_path': './data/ETT/', 'data_path': 'ETTh1.csv', 'features': 'M', 'target': 'OT', 'freq': 'h', 'seq_len': 96, 'label_len': 48, 'pred_len': 24, 'enc_in': 7, 'dec_in': 7, 'c_out': 7, 'd_model': 512, 'n_heads': 8, 'e_layers': 2, 'd_layers': 1, 'd_ff': 2048, 'factor': 5, 'distil': True, 'dropout': 0.05, 'attn': 'prob', 'embed': 'timeF', 'activation': 'gelu', 'output_attention': False, 'num_workers': 0, 'itr': 2, 'train_epochs': 6, 'batch_size': 32, 'patience': 3, 'learning_rate': 0.0001, 'des': 'test', 'loss': 'mse', 'lradj': 'type1', 'use_gpu': True, 'gpu